In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.8.0+cpu


In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

/home/lukas/git_repos/PytorchGeometricTutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Tutorial 6  
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

### Load the data

In [3]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_37973/2929005015.py:1: SyntaxWarning: invalid escape sequence '\.'
  dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
/tmp/ipykernel_37973/2929005015.py:2: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  dataset.data


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [4]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [5]:
data = train_test_split_edges(data)

/tmp/ipykernel_37973/3076777706.py:1: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  data = train_test_split_edges(data)


In [6]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [7]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [8]:
from torch_geometric.nn import GAE

In [9]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [10]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [11]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

/tmp/ipykernel_37973/2919475705.py:10: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  return float(loss)


Epoch: 001, AUC: 0.5983, AP: 0.6297
Epoch: 002, AUC: 0.6178, AP: 0.6466
Epoch: 003, AUC: 0.6251, AP: 0.6548
Epoch: 004, AUC: 0.6287, AP: 0.6590
Epoch: 005, AUC: 0.6302, AP: 0.6620
Epoch: 006, AUC: 0.6317, AP: 0.6642
Epoch: 007, AUC: 0.6327, AP: 0.6661
Epoch: 008, AUC: 0.6339, AP: 0.6683
Epoch: 009, AUC: 0.6347, AP: 0.6701
Epoch: 010, AUC: 0.6355, AP: 0.6721
Epoch: 011, AUC: 0.6356, AP: 0.6746
Epoch: 012, AUC: 0.6355, AP: 0.6768
Epoch: 013, AUC: 0.6351, AP: 0.6796
Epoch: 014, AUC: 0.6347, AP: 0.6828
Epoch: 015, AUC: 0.6340, AP: 0.6856
Epoch: 016, AUC: 0.6332, AP: 0.6891
Epoch: 017, AUC: 0.6321, AP: 0.6916
Epoch: 018, AUC: 0.6312, AP: 0.6942
Epoch: 019, AUC: 0.6300, AP: 0.6955
Epoch: 020, AUC: 0.6293, AP: 0.6963
Epoch: 021, AUC: 0.6289, AP: 0.6974
Epoch: 022, AUC: 0.6286, AP: 0.6983
Epoch: 023, AUC: 0.6285, AP: 0.6992
Epoch: 024, AUC: 0.6285, AP: 0.6997
Epoch: 025, AUC: 0.6284, AP: 0.7000
Epoch: 026, AUC: 0.6285, AP: 0.7006
Epoch: 027, AUC: 0.6290, AP: 0.7016
Epoch: 028, AUC: 0.6296, AP:

In [12]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-0.6096, -0.4565],
        [ 1.0281,  0.9120],
        [-0.7149, -0.6427],
        ...,
        [ 0.1671,  0.1444],
        [-0.7149, -0.6427],
        [-0.7149, -0.6427]], grad_fn=<AddBackward0>)

## Are the results (AUC) and (AP) easy to read and compare?

# Use Tensorboard

In [14]:
from torch.utils.tensorboard import SummaryWriter

In [15]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [16]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [17]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6140, AP: 0.6441
Epoch: 002, AUC: 0.6316, AP: 0.6597
Epoch: 003, AUC: 0.6345, AP: 0.6635
Epoch: 004, AUC: 0.6380, AP: 0.6682
Epoch: 005, AUC: 0.6394, AP: 0.6715
Epoch: 006, AUC: 0.6406, AP: 0.6747
Epoch: 007, AUC: 0.6418, AP: 0.6782
Epoch: 008, AUC: 0.6424, AP: 0.6802
Epoch: 009, AUC: 0.6418, AP: 0.6814
Epoch: 010, AUC: 0.6415, AP: 0.6834
Epoch: 011, AUC: 0.6405, AP: 0.6854
Epoch: 012, AUC: 0.6399, AP: 0.6885
Epoch: 013, AUC: 0.6391, AP: 0.6914
Epoch: 014, AUC: 0.6376, AP: 0.6940
Epoch: 015, AUC: 0.6365, AP: 0.6968
Epoch: 016, AUC: 0.6351, AP: 0.6991
Epoch: 017, AUC: 0.6336, AP: 0.7005
Epoch: 018, AUC: 0.6329, AP: 0.7016
Epoch: 019, AUC: 0.6323, AP: 0.7028
Epoch: 020, AUC: 0.6316, AP: 0.7035
Epoch: 021, AUC: 0.6315, AP: 0.7041
Epoch: 022, AUC: 0.6315, AP: 0.7048
Epoch: 023, AUC: 0.6320, AP: 0.7058
Epoch: 024, AUC: 0.6328, AP: 0.7067
Epoch: 025, AUC: 0.6338, AP: 0.7076
Epoch: 026, AUC: 0.6356, AP: 0.7089
Epoch: 027, AUC: 0.6383, AP: 0.7105
Epoch: 028, AUC: 0.6418, AP:

## Graph Variational AutoEncoder (GVAE)

In [18]:
from torch_geometric.nn import VGAE

In [19]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_37973/3226588792.py:1: SyntaxWarning: invalid escape sequence '\.'
  dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
/tmp/ipykernel_37973/3226588792.py:4: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  data = train_test_split_edges(data)


In [20]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [21]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [22]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6366, AP: 0.6729
Epoch: 002, AUC: 0.6427, AP: 0.6757
Epoch: 003, AUC: 0.6420, AP: 0.6757
Epoch: 004, AUC: 0.6414, AP: 0.6745
Epoch: 005, AUC: 0.6401, AP: 0.6739
Epoch: 006, AUC: 0.6394, AP: 0.6740
Epoch: 007, AUC: 0.6384, AP: 0.6740
Epoch: 008, AUC: 0.6376, AP: 0.6743
Epoch: 009, AUC: 0.6372, AP: 0.6748
Epoch: 010, AUC: 0.6349, AP: 0.6740
Epoch: 011, AUC: 0.6329, AP: 0.6732
Epoch: 012, AUC: 0.6283, AP: 0.6705
Epoch: 013, AUC: 0.6169, AP: 0.6613
Epoch: 014, AUC: 0.6000, AP: 0.6479
Epoch: 015, AUC: 0.5861, AP: 0.6364
Epoch: 016, AUC: 0.5618, AP: 0.6182
Epoch: 017, AUC: 0.5267, AP: 0.5866
Epoch: 018, AUC: 0.5142, AP: 0.5748
Epoch: 019, AUC: 0.5160, AP: 0.5777
Epoch: 020, AUC: 0.5417, AP: 0.6033
Epoch: 021, AUC: 0.5659, AP: 0.6198
Epoch: 022, AUC: 0.5775, AP: 0.6268
Epoch: 023, AUC: 0.5878, AP: 0.6384
Epoch: 024, AUC: 0.5998, AP: 0.6474
Epoch: 025, AUC: 0.6055, AP: 0.6514
Epoch: 026, AUC: 0.6154, AP: 0.6570
Epoch: 027, AUC: 0.6216, AP: 0.6612
Epoch: 028, AUC: 0.6254, AP: